In [1]:
import pandas as pd

In [2]:
import itertools

In [3]:
import glob

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import matplotlib.pyplot as plt

import numpy as np

In [6]:
def prepare_data(dimensions, files):
    
    columns = (['player index', 'co player index'] + [f'p{i}' for i in range(1, 17)] + 
               [f'q{i}' for i in range(1, 17)] + ['e', 'cc', 'cd', 'dc', 'dd'])
    
    dfs = [pd.read_csv(file, header=None, names=columns) for file in files]

    df = pd.concat(dfs)
    
    rhos = df[df['player index'] == df['co player index']]
    rhos['rho'] = 0
    for i, row in rhos.iterrows():
        rhos.loc[i, 'rho'] = row['cc'] + row['cd']
    df = df.merge(rhos[['player index', 'rho']], how='left', left_on='player index', right_on='player index')
    
    
    df['rho p'] = df.apply(lambda row: row['cc'] + row['cd'], axis=1)

    df['rho q'] = df.apply(lambda row: row['cc'] + row['dc'], axis=1)

    df['x'] = df.apply(lambda row: row['rho'] - row['rho p'], axis=1)

    df['y'] = df.apply(lambda row: row['rho'] - row['rho q'], axis=1)
    
    df['y / x'] = df['y'] / df['x']
    
    return df

In [7]:
def vaquero_method(df):
    sets = []
    for j, group in df.groupby('player index'):

        Q1, Q2, Q3, Q4 = [], [], [], []

        for i, row in group.iterrows():

            if row['player index'] != row['co player index']:

                if row['x'] > 0 and row['y'] > 0:
                    Q1.append(row['y'] / row['x'])

                if row['x'] < 0 and row['y'] <= 0:

                    Q2.append(row['y'] / row['x'])

                if row['x'] <= 0 and row['y'] > 0:
                    Q3.append(row['co player index'])

                if row['x'] == 0 and row['y'] == 0:

                    Q4.append(row['co player index'])

        sets.append([j, Q1, Q2, Q3, Q4])
    return sets

In [8]:
def payoffs_of_p(rho, b, c):
    return (b * rho) - (c * rho)

def payoffs_of_q(rho_p, rho_q, b, c):
    return (b * rho_p) - (c * rho_q)

### Memory-two (test results from Hilde 2017 paper)

In [10]:
dimensions = 2

files = glob.glob(f"../data/cooperation_data/memone/error_0.01_n_{dimensions}_*.csv")

In [12]:
memory_two = prepare_data(dimensions, files)

In [13]:
sets = vaquero_method(memory_two)

In [16]:
df = pd.DataFrame(sets, columns=['player index', 'Q1 (y / x)', 'Q2 (y / x)', 'Q3 (indices)', 'Q4 (indices)'])

In [18]:
Q3_empty = []
Q4_empty = []

for i, row in df.iterrows():
    Q3_empty.append(len(row['Q3 (indices)']) == 0)
    Q4_empty.append(len(row['Q4 (indices)']) == 0)

In [19]:
df['Q3 empty'] = Q3_empty
df['Q4 empty'] = Q4_empty

In [20]:
order_from_paper = [36874,32778,34954,39050,39322,35226,33050,37146,36880,61450,61456]

sorterIndex = dict(zip(order_from_paper, range(len(order_from_paper))))

In [21]:
df['player index'] = df['player index'].map(sorterIndex)

In [22]:
df = df.sort_values('player index')

In [23]:
for i, row in df[(df['Q3 empty'] == True) & (df['Q4 empty'] == True)].iterrows():
    ub = False
    lb = False
    
    if row['Q1 (y / x)']:
        lb = max(row['Q1 (y / x)'])
    if row['Q2 (y / x)']:
        ub = min(row['Q2 (y / x)'])
        
#     if lb <= ub:
    
    print(f"{row['player index']}: {round(lb, 2)} <= b/c <= {round(ub, 2)}")

0: 1.53 <= b/c <= 0
1: 1.53 <= b/c <= 0
2: 1.53 <= b/c <= 0
3: 1.53 <= b/c <= 0
4: 2.04 <= b/c <= 0
5: 2.04 <= b/c <= 0
6: 2.04 <= b/c <= 105.32
7: 2.04 <= b/c <= 0
8: 2.08 <= b/c <= 0
9: 2.02 <= b/c <= 0
10: 2.04 <= b/c <= 0


### One bit

In [36]:
dimensions = 1

files = glob.glob(f"../data/cooperation_data/error_0.01_n_{dimensions}_*.csv")

In [37]:
one_bit = prepare_data(dimensions, files)

In [38]:
sets = vaquero_method(one_bit)

In [39]:
pd.DataFrame(sets, columns=['player index', 'Q1 (y / x)', 'Q2 (y / x)', 'Q3 (indices)', 'Q4 (indices)'])

,player index,Q1 (y / x),Q2 (y / x),Q3 (indices),Q4 (indices)
0,1,[],[],[],[]
1,2,[],[],[],[]
2,3,[],[],[],[]
3,4,[],[],[],[]
4,5,[],[],[],[]
5,6,[],[],[],[]
6,7,[],[],[],[]
7,8,[],[],[],[]
8,9,[],[],[],[]
9,10,[],[],[],[]


### Two bits

In [ ]:
dimensions = 4
files = glob.glob(f"../data/cooperation_data/error_0.01_n_{dimensions}_*.csv")

In [ ]:
two_bit = prepare_data(dimensions, files)

In [14]:
sets = vaquero_method(two_bit)

In [15]:
df = pd.DataFrame(sets, columns=['player index', 'Q1 (y / x)', 'Q2 (y / x)', 'Q3 (indices)', 'Q4 (indices)'])


In [16]:
Q3_empty = []
Q4_empty = []

for i, row in df.iterrows():
    Q3_empty.append(len(row['Q3 (indices)']) == 0)
    Q4_empty.append(len(row['Q4 (indices)']) == 0)

In [17]:
df['Q3 empty'] = Q3_empty
df['Q4 empty'] = Q4_empty

In [22]:
for i, row in df[(df['Q3 empty'] == True) & (df['Q4 empty'] == True)].iterrows():
    ub = False
    lb = False
    
    if row['Q1 (y / x)']:
        lb = max(row['Q1 (y / x)'])
    if row['Q2 (y / x)']:
        ub = min(row['Q2 (y / x)'])
        
#     if lb <= ub:
    
    print(f"{row['player index']}: {round(lb, 2)} <= b/c <= {round(ub, 2)}")

1: 0 <= b/c <= 0
3: 1.04 <= b/c <= 0
5: 1.04 <= b/c <= 2882353642.71
9: 19.5 <= b/c <= 1.02
15: 51.9 <= b/c <= 19.5


### Three bits

In [21]:
three_bit = prepare_data(8)

In [127]:
three_bit = three_bit.sort_values('player index')

In [128]:
sets = vaquero_method(three_bit)

In [129]:
df = pd.DataFrame(sets, columns=['player index', 'Q1 (y / x)', 'Q2 (y / x)', 'Q3 (indices)', 'Q4 (indices)'])

In [130]:
Q3_empty = []
Q4_empty = []

for i, row in df.iterrows():
    Q3_empty.append(len(row['Q3 (indices)']) == 0)
    Q4_empty.append(len(row['Q4 (indices)']) == 0)

In [131]:
df['Q3 empty'] = Q3_empty
df['Q4 empty'] = Q4_empty

In [133]:
for i, row in df[(df['Q3 empty'] == True) & (df['Q4 empty'] == True)].iterrows():
    ub = False
    lb = False
    
    if row['Q1 (y / x)']:
        lb = max(row['Q1 (y / x)'])
    if row['Q2 (y / x)']:
        ub = min(row['Q2 (y / x)'])
    if lb <= ub:
    
        print(f"{row['player index']}: {lb} <= b/c <= {ub}")

1: False <= b/c <= False
35: 1.0412328199480185 <= b/c <= 1.0424536300792253
69: 1.0412328408761917 <= b/c <= 1.0436048213156999


### Four bits

In [57]:
four_bit = prepare_data(16)

In [58]:
sets = vaquero_method(four_bit)

In [59]:
df = pd.DataFrame(sets, columns=['player index', 'Q1 (y / x)', 'Q2 (y / x)', 'Q3 (indices)', 'Q4 (indices)'])

In [60]:
Q3_empty = []
Q4_empty = []

for i, row in df.iterrows():
    Q3_empty.append(len(row['Q3 (indices)']) == 0)
    Q4_empty.append(len(row['Q4 (indices)']) == 0)

In [61]:
df['Q3 empty'] = Q3_empty
df['Q4 empty'] = Q4_empty

In [65]:
indices = []
lbounds = []
ubounds = []
for i, row in df[(df['Q3 empty'] == True) & (df['Q4 empty'] == True)].iterrows():
    ub = False
    lb = False
    
    if row['Q1 (y / x)']:
        lb = max(row['Q1 (y / x)'])
    if row['Q2 (y / x)']:
        ub = min(row['Q2 (y / x)'])
        
    if lb <= ub:
    
        print(f"{row['player index']}: {lb} <= b/c <= {ub}", f"-{lb <= ub}")
        indices.append(row['player index'])
        lbounds.append(round(lb, 3))
        ubounds.append(round(ub, 3))

1: False <= b/c <= 485101.21231723594 -True
35: 1.0414495046715793 <= b/c <= 1.0415829926006013 -True
73: 1.0419170119230037 <= b/c <= 1.047882076486478 -True
163: 1.0413538739633985 <= b/c <= 1.0415574235985448 -True
205: 1.0412328937089776 <= b/c <= 1.0426671633511733 -True
209: 1.0867506256938408 <= b/c <= 1.789273796073751 -True


In [181]:
columns = [f'p{i}' for i in range(1, 17)] + ['rho']

table = four_bit[(four_bit['player index'].isin(indices))][columns].drop_duplicates().reset_index(drop=True).round(3)

In [183]:
table['lb'] = lbounds
table['ub'] = ubounds